# Metodo de Montecarlo

In [ ]:
import sys
sys.path.append('..')
#Para poder leer los modulos dentro del directorio 
from Codigo.opcion_europea_bs import opcion_europea_bs
from Codigo.opcion_europea_mc import opcion_europea_mc

import numpy as np
import math

opcion_europea_mc
Def
    Calculador del precio de una opcion Europea con el modelo de MonteCarlo
Inputs
    - tipo : string - Tipo de contrato entre ["CALL","PUT"]
    - S : float - Spot price del activo
    - K : float - Strike price del contrato
    - T : float - Tiempo hasta la expiracion (en años)
    - r : float - Tasa 'libre de riesgo' (anualizada)
    - sigma : float - Volatilidad implicita (anualizada)
    - div : float - Tasa de dividendos continuos (anualizada)
    - pasos : int - Cantidad de caminos de montecarlo
Outputs
    - precio_MC: float - Precio del contrato


In [ ]:
#Inicializo los parametros

tipo = "CALL"
S = 100
K = 100
T = 1
r = 0.05
sigma = 0.25
div = 0

In [ ]:
#Tomo un vector de normales aleatorias
pasos = 20000 
z = np.random.normal(0,1,pasos)
#z


In [ ]:
opcion = np.zeros(pasos)

In [ ]:
for i in range(0,pasos):
    if tipo == "CALL":
        payoff = max( 0 , S * math.exp((r-div - 0.5 * math.pow(sigma,2)) * T + sigma * math.sqrt(T)  * z[i]) - K)
    elif tipo == "PUT":
        payoff = max(0, K - S * math.exp((r-div - 0.5 * math.pow(sigma, 2)) * T + sigma * math.sqrt(T) * z[i]) )
    opcion[i] = math.exp(-r * T) * payoff


In [ ]:
precio_MC = np.mean(opcion)

precio_MC

### Convergencia del metodo

In [ ]:
tipo = "PUT"
S = 100
K = 100
T = 1
r = 0.05
sigma= 0.25
div =0.2

pasos_vec = [1,2,3,4,5,6,7,8,9,10,12,14,16,18,20,25,30,40,50,75,100,125,150,200,250,300, 500]      
precios = np.zeros(len(pasos_vec))

for index in range(len(pasos_vec)):
    precios[index] = opcion_europea_mc(tipo, S, K, T, r, sigma, div, pasos_vec[index])
 
precio_BS = opcion_europea_bs(tipo, S, K, T, r, sigma, div)

In [ ]:
from matplotlib import pyplot as plt
plt.axhline(y=precio_BS, color = 'black', linestyle='--', label='Precio BlackSholes')
plt.plot(pasos_vec,precios,'*', label='Precio Binomial')

plt.legend()

plt.xlabel('Pasos del arbol Binomial')
plt.ylabel('Precio de la opcion')
plt.title('Precio de una opcion a tiempo inicial con respecto al numero de pasos del arbol')

plt.show()

# Metodo de diferencias finitas

In [ ]:
from Codigo.opcion_americana_fd import opcion_americana_fd
from Codigo.opcion_europea_fd import opcion_europea_fd

from scipy.interpolate import interp1d

opcion_europea_fd
Def
    Calculador del precio de una opcion Europea con el modelo de Diferencias Finitas (metodo explicito)
Inputs
    - tipo : string - Tipo de contrato entre ["CALL","PUT"]
    - S : float - Spot price del activo
    - K : float - Strike price del contrato
    - T : float - Tiempo hasta la expiracion (en años)
    - r : float - Tasa 'libre de riesgo' (anualizada)
    - sigma : float - Volatilidad implicita (anualizada)
    - div : float - Tasa de dividendos continuos (anualizada)
Outputs
    - precio_FD: float - Precio del contrato

In [ ]:
tipo = "CALL"
S = 100
K = 100
T = 1
r = 0.05
sigma= 0.25
div =0.0

In [ ]:
#Hadrcode de la grilla de diferencias finitas
M = 160
N = 1600
dS = 2 * S / M
dt = T / N

In [ ]:
# Grilla de spots y tiempos
S_vec = np.linspace(0, 2*S, M+1)
t_vec = np.linspace(0, T, N+1)

In [ ]:
# Armado de la matriz tridiagonal
j = np.arange(1,M)
j2 = np.zeros(M-1)
aj = np.zeros(M-1)
bj = np.zeros(M-1)
cj = np.zeros(M-1)

for index in range(0,M-1):
    sigma2 = sigma*sigma
    j2[index] = j[index] * j[index]
    aj[index] = 0.5 * dt * (sigma2 * j2[index]- (r-div) * j[index])
    bj[index] = 1-dt * (sigma2 * j2[index] + r)
    cj[index] = 0.5 * dt * (sigma2 * j2[index] + (r-div) * j[index])

In [ ]:
# Matriz tridiagonal

A = np.diag(bj)
for index in range(0, M - 2):
    A[index + 1, index] = aj[index + 1]  # terms below the diagonal
    A[index, index + 1] = cj[index]  # terms above the diagonal

In [ ]:
# Matriz de precios de la opcion
opcion_precios = np.zeros((M+1,N+1))

In [ ]:
#Condiciones de contorno

# Condicion final - Payoff

if tipo == "CALL":
    opcion_precios[:,-1] = np.maximum(S_vec - K, 0)
elif tipo == "PUT":
    opcion_precios[:,-1] = np.maximum(K - S_vec, 0)

In [ ]:
# Casos limite en S=0 y S~inf

if tipo == "CALL":
    opcion_precios[0, :] = 0
    opcion_precios[-1, :] = S_vec[-1]*np.exp(-div*np.flip(t_vec)) - K * np.exp(-r*np.flip(t_vec))
elif tipo == "PUT":
    opcion_precios[0, :] = K * np.exp(-r * np.flip(t_vec))
    opcion_precios[-1, :] = 0 #K * np.exp(-r * np.flip(t_vec))

In [ ]:
    
# Calculo en el interior
# variable auxiliar para sumar en la primer y ultimo fila
constantes_bordes = np.array((aj[0], cj[-1]))
for i in list(reversed(range(0,N))):

    opcion_precios[1:M,i] = A @ opcion_precios[1:M,i+1]
    #Offset the first and last terms
    opcion_precios[[1,M-1],i] = opcion_precios[[1,M-1],i] + constantes_bordes * opcion_precios[[0, M],i+1];

In [ ]:
#En este punto ya esta TODA la grilla, ahora calculo lo requerido

f = interp1d(S_vec,opcion_precios[:,0])
precio_FD = float(f(S))

precio_FD


In [ ]:
from mpl_toolkits import mplot3d

X, Y = np.meshgrid(t_vec, S_vec)
Z = opcion_precios

In [ ]:
print('X', np.shape(X))
print('Y', np.shape(Y))
print('Z', np.shape(Z))

In [ ]:
fig = plt.figure()

ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z, rstride=1, cstride=1,
                cmap='winter', edgecolor='none')

ax.set_title('surface');
ax.set_xlabel('Tiempo - t')
ax.set_ylabel('Precio del subyacente - S')
ax.set_zlabel('Precio del Derivado')

plt.show()

In [ ]:
#import mayavi.mlab as mlab

#s = mlab.mesh(X, Y, Z)
#alpha = 30  # degrees
#mlab.view(azimuth=0, elevation=90, roll=-90+alpha)

#mlab.show()

# QuantLib o Introduccion a una Verdadera plataforma de pricing/riesgo

In [ ]:
from QuantLib import *
#%matplotlib inline

from Codigo.utils_plots import *

### Opciones

### Supongamos una opcion de AAPL con precio strike de 450 expirando el 16 de octubre de 2020. Supongamos que el spot es 440. LA vol implicita es sabida que es 31%, y tiene un dividendo de 0.75%. Valuemos la opcion al dia de hoy (10 de Agosto 2020)

#### Definimos los inputs

In [ ]:
S = 456.30
K = 460
sigma = 0.3394 # the historical vols for a year
div =  0.075
option_type = Option.Call
r = 0.0013

#### Para las fechas QuantLib es mas preciso. Pide convenciones y calendarios

In [ ]:
maturity_date = Date(16, 10, 2020)
calculation_date = Date(6, 8, 2020)
day_count = Actual365Fixed()
calendar = UnitedStates()
Settings.instance().evaluationDate = calculation_date

#### Aqui construimos a la opcion europea

In [ ]:
payoff = PlainVanillaPayoff(option_type, K)
exercise = EuropeanExercise(maturity_date)
european_option = VanillaOption(payoff, exercise)

#### Aca constuimos el proceso de Black Sholes Merton

##### El objeto spot

In [ ]:
spot_obj = QuoteHandle(SimpleQuote(S))

##### El objeto curva de descuento (flat en este caso, constante)

In [ ]:
rate_obj = YieldTermStructureHandle(FlatForward(calculation_date, 
                                                            r, 
                                                    day_count))

##### El objeto curva de dividendos (flat en este caso, constante)

In [ ]:
dividend_obj = YieldTermStructureHandle(FlatForward(calculation_date, 
                                                      div, 
                                                      day_count))

##### El objeto volatilidad (flat en este caso, constante)

In [ ]:
vol_obj = BlackVolTermStructureHandle(BlackConstantVol(calculation_date, 
                                                                 calendar, 
                                                                 sigma, 
                                                             day_count))

In [ ]:
##### El proceso propiamente dicho

In [ ]:
bsm_process = BlackScholesMertonProcess(spot_obj, 
                                        dividend_obj, 
                                        rate_obj, 
                                        vol_obj)

## Modelos de precio

###  `AnalyticEuropeanEngine` (Black Scholes)

In [ ]:
european_option.setPricingEngine(AnalyticEuropeanEngine(bsm_process))

bs_price = european_option.NPV()
print("El precio teorico usando el modelo de BS es: ", bs_price)

###  `FdBlackScholesVanillaEngine` (Diferencias Finitas)

In [ ]:
european_option.setPricingEngine(FdBlackScholesVanillaEngine(bsm_process))

fd_price = european_option.NPV()
print("El precio teorico usando el modelo de Dif. Finitas es: ", fd_price)

###  `MCEuropeanEngine` (Montecarlo)

In [ ]:
european_option.setPricingEngine(MCEuropeanEngine(bsm_process, "PseudoRandom", timeSteps=20, requiredSamples=500000))

mc_price = european_option.NPV()
print("El precio teorico usando el modelo de MonteCarlo es: ",  mc_price)

###  `BinomialVanillaEngine` (Binomial)

In [ ]:
european_option.setPricingEngine(BinomialVanillaEngine(bsm_process, "crr", 1000))

bin_price = european_option.NPV()
print("El precio teorico usando el modelo Binomial es: ",  bin_price)

# Comparacion de TODOS* los modelos vistos

In [ ]:
S = 100
K = 100
sigma = 0.25 # the historical vols for a year
div =  0.0
r = 0.05

tipo = 'CALL'

option_type = Option.Call

maturity_date = Date(10, 8, 2021)
calculation_date = Date(10, 8, 2020)
day_count = Actual365Fixed()
calendar = UnitedStates()
Settings.instance().evaluationDate = calculation_date

T = 1



## Pricers vistos en clase

In [ ]:
from Codigo.opcion_europea_bs import opcion_europea_bs
from Codigo.opcion_europea_mc import opcion_europea_mc
from Codigo.opcion_americana_bin import opcion_americana_bin
from Codigo.opcion_americana_fd import opcion_americana_fd
from Codigo.opcion_europea_bin_c import opcion_europea_bin_c
from Codigo.opcion_europea_fd import opcion_europea_fd
from Codigo.opcion_europea_bin import opcion_europea_bin


In [ ]:
precio_bs = opcion_europea_bs(tipo, S, K, T, r, sigma, div)
precio_bin = opcion_europea_bin(tipo, S, K, T, r, sigma, div, 1000)
precio_mc = opcion_europea_mc(tipo, S, K, T, r, sigma, div, 10000)
precio_fd = opcion_europea_fd(tipo, S, K, T, r, sigma, div)


## Precios QuantLib

In [ ]:
payoff = PlainVanillaPayoff(option_type, K)
exercise = EuropeanExercise(maturity_date)
european_option = VanillaOption(payoff, exercise)

spot_obj = QuoteHandle(SimpleQuote(S))

rate_obj = YieldTermStructureHandle(FlatForward(calculation_date, 
                                                            r, 
                                                    day_count))

dividend_obj = YieldTermStructureHandle(FlatForward(calculation_date, 
                                                      div, 
                                                      day_count))

vol_obj = BlackVolTermStructureHandle(BlackConstantVol(calculation_date, 
                                                                 calendar, 
                                                                 sigma, 
                                                             day_count))

bsm_process = BlackScholesMertonProcess(spot_obj, 
                                        dividend_obj, 
                                        rate_obj, 
                                        vol_obj)



In [ ]:
european_option.setPricingEngine(AnalyticEuropeanEngine(bsm_process))
bs_price = european_option.NPV()

european_option.setPricingEngine(FdBlackScholesVanillaEngine(bsm_process))
fd_price = european_option.NPV()

european_option.setPricingEngine(MCEuropeanEngine(bsm_process, "PseudoRandom", timeSteps=20, requiredSamples=500000))
mc_price = european_option.NPV()

european_option.setPricingEngine(BinomialVanillaEngine(bsm_process, "crr", 1000))
bin_price = european_option.NPV()




In [ ]:
print("Precio modelo BS visto en clase:", precio_bs)
print("Precio modelo Binomial visto en clase:", precio_bin)
print("Precio modelo Montecarlo visto en clase:", precio_mc)
print("Precio modelo Dif. Finitas visto en clase:", precio_fd)


print("Precio modelo BS QuantLib", bs_price)
print("Precio modelo Binomial QuantLib:", bin_price)
print("Precio modelo Montecarlo QuantLib:", mc_price)
print("Precio modelo Dif. FinitasQuantLib:", fd_price)
